# FCN

Fully Connected Network

## GPU Check

In [1]:
# # check available GPU in terminal: watch -d -n 0.5 nvidia-smi
# # select one GPU to train on:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

## Imports

In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../modules/'))
sys.path.insert(1, os.path.join(sys.path[0], '../data_creation/'))
sys.path.insert(1, os.path.join(sys.path[0], '../networks/'))
sys.path.insert(1, os.path.join(sys.path[0], '../network_trainers/'))

from FNN_BN_R import FNN_BN_R
from FNN_BN_R_Trainer import FNN_BN_R_Trainer
from Data_Creator_R import Data_Creator_R
from data_manipulation import *

## Load and Setup Data

In [2]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

In [2]:
# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

***
## A

In [9]:
FNN_BN_R_A = FNN_BN_R('FNN_BN_R_A',
                      layer_nodes=[512,256,128,128,64,64,32],
                      adam_initial_learning_rate = 0.001)

In [10]:
FNN_BN_R_A.print_params()

{'accuracy_threshold': 0.00625,
 'adam_initial_learning_rate': 0.001,
 'cost': 'MSE',
 'gaussian_shift_scalar': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'FNN_BN_R_A'}


In [11]:
trainer = FNN_BN_R_Trainer(FNN_BN_R_A,
                           Data_Creator_R,
                           num_epochs=1000,
                           batch_size=32)

In [12]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses=100)

In [13]:
trainer.print_params()

{'batch_size': 32,
 'fcl_keep_prob': 0.9,
 'log_dir': 'logs/',
 'metric_names': ['MISGs', 'MSEs', 'MQEs', 'PWTs'],
 'model_save_interval': 25,
 'name': 'FNN_BN_R_A',
 'num_epochs': 1000,
 'pretrained_model_path': None,
 'sample_keep_prob': 0.8}


In [14]:
trainer.train()

saving metrics - saved

***
## B

In [23]:
FNN_BN_R_B = FNN_BN_R('FNN_BN_R_B',
                      layer_nodes=[])
FNN_BN_R_B.load_params('logs/FNN_BN_R_A/params/FNN_BN_R')

params loaded

In [24]:
FNN_BN_R_B.cost = 'MISG'
FNN_BN_R_B.adam_initial_learning_rate = 1e-05
FNN_BN_R_B.print_params()

{'accuracy_threshold': 0.00625,
 'adam_initial_learning_rate': 1e-05,
 'cost': 'MISG',
 'gaussian_shift_scalar': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'FNN_BN_R_B'}


In [25]:
trainer = FNN_BN_R_Trainer(FNN_BN_R_B,
                           Data_Creator_R,
                           num_epochs=1000,
                           batch_size=64)
trainer.pretrained_model_path = 'logs/FNN_BN_R_A/trained_model.ckpt-349'

In [26]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses=100)

In [27]:
trainer.print_params()

{'batch_size': 64,
 'fcl_keep_prob': 0.9,
 'log_dir': 'logs/',
 'metric_names': ['MISGs', 'MSEs', 'MQEs', 'PWTs'],
 'model_save_interval': 25,
 'name': 'FNN_BN_R_B',
 'num_epochs': 1000,
 'pretrained_model_path': 'logs/FNN_BN_R_A/trained_model.ckpt-349',
 'sample_keep_prob': 0.8}


In [28]:
trainer.train()

Network Ready - 721249 trainable parametersINFO:tensorflow:Restoring parameters from logs/FNN_BN_R_A/trained_model.ckpt-349
saving metrics - saved

***
## C

In [45]:
FNN_BN_R_C = FNN_BN_R('FNN_BN_R_C',
                      layer_nodes=[])
FNN_BN_R_C.load_params('logs/FNN_BN_R_B/params/FNN_BN_R')

params loaded

In [46]:
FNN_BN_R_C.cost = 'MISG'
FNN_BN_R_C.adam_initial_learning_rate = 1e-04
FNN_BN_R_C.print_params()

{'accuracy_threshold': 0.00625,
 'adam_initial_learning_rate': 0.0001,
 'cost': 'MISG',
 'gaussian_shift_scalar': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'FNN_BN_R_C'}


In [47]:
trainer = FNN_BN_R_Trainer(FNN_BN_R_C,
                           Data_Creator_R,
                           num_epochs=1000,
                           batch_size=256)
trainer.pretrained_model_path = 'logs/FNN_BN_R_B/trained_model.ckpt-199'

In [48]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses=100)

In [49]:
trainer.print_params()

{'batch_size': 256,
 'fcl_keep_prob': 0.9,
 'log_dir': 'logs/',
 'metric_names': ['MISGs', 'MSEs', 'MQEs', 'PWTs'],
 'model_save_interval': 25,
 'name': 'FNN_BN_R_C',
 'num_epochs': 1000,
 'pretrained_model_path': 'logs/FNN_BN_R_B/trained_model.ckpt-199',
 'sample_keep_prob': 0.8}


In [50]:
trainer.train()

Network Ready - 721249 trainable parametersINFO:tensorflow:Restoring parameters from logs/FNN_BN_R_B/trained_model.ckpt-199
saving metrics - saved